In [ ]:
%%capture
from typing import *
from fastai2.basics import *
from fastai2.text.all import *
from fastai2.callback import *

from transformers import AutoConfig, AutoTokenizer, AutoModelWithLMHead, GPT2LMHeadModel, GPT2Tokenizer
from fastai_transformers_utils.general import TransformersTokenizer, TransformersNumericalize, GPT2LMHeadCallback, gpt2_lmhead_split

In [ ]:
# all_slow

# Example: GPT2 Language Model on WikiText_Tiny
> This is also an integration test.

In [ ]:
model_name = 'gpt2'
seq_len = 72
bs = 64

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Data and Tokenization

In [ ]:
path = untar_data(URLs.WIKITEXT_TINY)
df_train = pd.read_csv(path/'train.csv', header=None, names=['text'])
df_valid = pd.read_csv(path/'test.csv', header=None, names=['text'])
df_all = pd.concat([df_train, df_valid])
df_all.head()

,text
0,"\n = 2013 – 14 York City F.C. season = \n \n The 2013 – 14 season was the <unk> season of competitive association football and 77th season in the Football League played by York City Football Club , a professional football club based in York , North Yorkshire , England . Their 17th @-@ place finish in 2012 – 13 meant it was their second consecutive season in League Two . The season ran from 1 July 2013 to 30 June 2014 . \n Nigel Worthington , starting his first full season as York manager , made eight permanent summer signings . By the turn of the year York were only above the relegation z..."
1,"\n = Big Boy ( song ) = \n \n "" Big Boy "" <unk> "" I 'm A Big Boy Now "" was the first single ever recorded by the Jackson 5 , which was released by Steeltown Records in January 1968 . The group played instruments on many of their Steeltown compositions , including "" Big Boy "" . The song was neither a critical nor commercial success , but the Jackson family were delighted with the outcome nonetheless . \n The Jackson 5 would release a second single with Steeltown Records before moving to Motown Records . The group 's recordings at Steeltown Records were thought to be lost , but they were re..."
2,"\n = The Remix ( Lady Gaga album ) = \n \n The Remix is a remix album by American recording artist Lady Gaga . Released in Japan on March 3 , 2010 , it contains remixes of the songs from her first studio album , The Fame ( 2008 ) , and her third extended play , The Fame Monster ( 2009 ) . A revised version of the track list was prepared for release in additional markets , beginning with Mexico on May 3 , 2010 . A number of recording artists have produced the songs , including Pet Shop Boys , Passion Pit and The Sound of Arrows . The remixed versions feature both uptempo and <unk> composit..."
3,"\n = New Year 's Eve ( Up All Night ) = \n \n "" New Year 's Eve "" is the twelfth episode of the first season of the American comedy television series Up All Night . The episode originally aired on NBC in the United States on January 12 , 2012 . It was written by Erica <unk> and was directed by Beth McCarthy @-@ Miller . The episode also featured a guest appearance from Jason Lee as Chris and Reagan 's neighbor and Ava 's boyfriend , Kevin . \n During Reagan ( Christina Applegate ) and Chris 's ( Will <unk> ) first New Year 's Eve game night , Reagan 's competitiveness comes out causing Ch..."
4,"\n = Geopyxis carbonaria = \n \n Geopyxis carbonaria is a species of fungus in the genus Geopyxis , family <unk> . First described to science in 1805 , and given its current name in 1889 , the species is commonly known as the charcoal loving elf @-@ cup , dwarf <unk> cup , <unk> <unk> cup , or pixie cup . The small , <unk> @-@ shaped fruitbodies of the fungus are reddish @-@ brown with a whitish fringe and measure up to 2 cm ( 0 @.@ 8 in ) across . They have a short , tapered stalk . Fruitbodies are commonly found on soil where brush has recently been burned , sometimes in great numbers ...."


In [ ]:
tok_list = L(parallel_gen(TransformersTokenizer, df_all.text, tokenizer=tokenizer)
            ).sorted().itemgot(1)
tok_df = df_all.copy()
tok_df.text =  tok_list.map(lambda x: ' '.join(x))
tok_df.head()

,text
0,"= Ġ2013 ĠâĢĵ Ġ14 ĠYork ĠCity ĠF . C . Ġseason Ġ= Ġ Ċ Ġ Ċ ĠThe Ġ2013 ĠâĢĵ Ġ14 Ġseason Ġwas Ġthe Ġ< unk > Ġseason Ġof Ġcompetitive Ġassociation Ġfootball Ġand Ġ77 th Ġseason Ġin Ġthe ĠFootball ĠLeague Ġplayed Ġby ĠYork ĠCity ĠFootball ĠClub Ġ, Ġa Ġprofessional Ġfootball Ġclub Ġbased Ġin ĠYork Ġ, ĠNorth ĠYorkshire Ġ, ĠEngland Ġ. ĠTheir Ġ17 th Ġ@ - @ Ġplace Ġfinish Ġin Ġ2012 ĠâĢĵ Ġ13 Ġmeant Ġit Ġwas Ġtheir Ġsecond Ġconsecutive Ġseason Ġin ĠLeague ĠTwo Ġ. ĠThe Ġseason Ġran Ġfrom Ġ1 ĠJuly Ġ2013 Ġto Ġ30 ĠJune Ġ2014 Ġ. Ġ Ċ ĠNigel ĠWor thing ton Ġ, Ġstarting Ġhis Ġfirst Ġfull Ġseason Ġas ĠYork Ġman..."
1,"= ĠBig ĠBoy Ġ( Ġsong Ġ) Ġ= Ġ Ċ Ġ Ċ Ġ"" ĠBig ĠBoy Ġ"" Ġ< unk > Ġ"" ĠI Ġ' m ĠA ĠBig ĠBoy ĠNow Ġ"" Ġwas Ġthe Ġfirst Ġsingle Ġever Ġrecorded Ġby Ġthe ĠJackson Ġ5 Ġ, Ġwhich Ġwas Ġreleased Ġby ĠSte elt own ĠRecords Ġin ĠJanuary Ġ1968 Ġ. ĠThe Ġgroup Ġplayed Ġinstruments Ġon Ġmany Ġof Ġtheir ĠSte elt own Ġcompositions Ġ, Ġincluding Ġ"" ĠBig ĠBoy Ġ"" Ġ. ĠThe Ġsong Ġwas Ġneither Ġa Ġcritical Ġnor Ġcommercial Ġsuccess Ġ, Ġbut Ġthe ĠJackson Ġfamily Ġwere Ġdelighted Ġwith Ġthe Ġoutcome Ġnonetheless Ġ. Ġ Ċ ĠThe ĠJackson Ġ5 Ġwould Ġrelease Ġa Ġsecond Ġsingle Ġwith ĠSte elt own ĠRecords Ġbefore Ġmoving Ġto ĠMot..."
2,"= ĠThe ĠRemix Ġ( ĠLady ĠGaga Ġalbum Ġ) Ġ= Ġ Ċ Ġ Ċ ĠThe ĠRemix Ġis Ġa Ġremix Ġalbum Ġby ĠAmerican Ġrecording Ġartist ĠLady ĠGaga Ġ. ĠReleased Ġin ĠJapan Ġon ĠMarch Ġ3 Ġ, Ġ2010 Ġ, Ġit Ġcontains Ġremix es Ġof Ġthe Ġsongs Ġfrom Ġher Ġfirst Ġstudio Ġalbum Ġ, ĠThe ĠFame Ġ( Ġ2008 Ġ) Ġ, Ġand Ġher Ġthird Ġextended Ġplay Ġ, ĠThe ĠFame ĠMonster Ġ( Ġ2009 Ġ) Ġ. ĠA Ġrevised Ġversion Ġof Ġthe Ġtrack Ġlist Ġwas Ġprepared Ġfor Ġrelease Ġin Ġadditional Ġmarkets Ġ, Ġbeginning Ġwith ĠMexico Ġon ĠMay Ġ3 Ġ, Ġ2010 Ġ. ĠA Ġnumber Ġof Ġrecording Ġartists Ġhave Ġproduced Ġthe Ġsongs Ġ, Ġincluding ĠPet ĠShop ĠBoys Ġ,..."
3,"= ĠNew ĠYear Ġ' s ĠEve Ġ( ĠUp ĠAll ĠNight Ġ) Ġ= Ġ Ċ Ġ Ċ Ġ"" ĠNew ĠYear Ġ' s ĠEve Ġ"" Ġis Ġthe Ġtw elfth Ġepisode Ġof Ġthe Ġfirst Ġseason Ġof Ġthe ĠAmerican Ġcomedy Ġtelevision Ġseries ĠUp ĠAll ĠNight Ġ. ĠThe Ġepisode Ġoriginally Ġaired Ġon ĠNBC Ġin Ġthe ĠUnited ĠStates Ġon ĠJanuary Ġ12 Ġ, Ġ2012 Ġ. ĠIt Ġwas Ġwritten Ġby ĠErica Ġ< unk > Ġand Ġwas Ġdirected Ġby ĠBeth ĠMcCarthy Ġ@ - @ ĠMiller Ġ. ĠThe Ġepisode Ġalso Ġfeatured Ġa Ġguest Ġappearance Ġfrom ĠJason ĠLee Ġas ĠChris Ġand ĠReagan Ġ' s Ġneighbor Ġand ĠAv a Ġ' s Ġboyfriend Ġ, ĠKevin Ġ. Ġ Ċ ĠDuring ĠReagan Ġ( ĠChristina ĠApp legate Ġ) Ġand ..."
4,"= ĠGe opy x is Ġcarbon aria Ġ= Ġ Ċ Ġ Ċ ĠGe opy x is Ġcarbon aria Ġis Ġa Ġspecies Ġof Ġfungus Ġin Ġthe Ġgenus ĠGe opy x is Ġ, Ġfamily Ġ< unk > Ġ. ĠFirst Ġdescribed Ġto Ġscience Ġin Ġ18 05 Ġ, Ġand Ġgiven Ġits Ġcurrent Ġname Ġin Ġ1889 Ġ, Ġthe Ġspecies Ġis Ġcommonly Ġknown Ġas Ġthe Ġcharcoal Ġloving Ġelf Ġ@ - @ Ġcup Ġ, Ġdwarf Ġ< unk > Ġcup Ġ, Ġ< unk > Ġ< unk > Ġcup Ġ, Ġor Ġp ixie Ġcup Ġ. ĠThe Ġsmall Ġ, Ġ< unk > Ġ@ - @ Ġshaped Ġfruit b odies Ġof Ġthe Ġfungus Ġare Ġredd ish Ġ@ - @ Ġbrown Ġwith Ġa Ġwhit ish Ġfringe Ġand Ġmeasure Ġup Ġto Ġ2 Ġcm Ġ( Ġ0 Ġ@ . @ Ġ8 Ġin Ġ) Ġacross Ġ. ĠThey Ġhave Ġa Ġsho..."


In [ ]:
# tok_df.to_csv(f'{model_name}_tok.csv', index=False)

In [ ]:
# tok_df = pd.read_csv(f'{model_name}_tok.csv')

## Databunch

In [ ]:
splits = [list(range(0, len(df_train))), list(range(len(df_train), len(df_all)))]
ds_tfms = [
    [attrgetter('text'), lambda x: x.split(' '), TransformersNumericalize(tokenizer)]
]
dsrc = DataSource(tok_df, tfms=ds_tfms, splits=splits)
dsrc.train[0], dsrc.decode(dsrc.train[0])

((tensor([   28,  2211,   784,  ...,    29,   784, 19530]),),
 ("= 2013 – 14 York City F.C. season = \n \n The 2013 – 14 season was the <unk> season of competitive association football and 77th season in the Football League played by York City Football Club, a professional football club based in York, North Yorkshire, England. Their 17th @-@ place finish in 2012 – 13 meant it was their second consecutive season in League Two. The season ran from 1 July 2013 to 30 June 2014. \n Nigel Worthington, starting his first full season as York manager, made eight permanent summer signings. By the turn of the year York were only above the relegation zone on goal difference, before a 17 @-@ match unbeaten run saw the team finish in seventh @-@ place in the 24 @-@ team 2013 – 14 Football League Two. This meant York qualified for the play @-@ offs, and they were eliminated in the semi @-@ final by Fleetwood Town. York were knocked out of the 2013 – 14 FA Cup, Football League Cup and Football League 

In [ ]:
def get_databunch(dsrc, bs: int, seq_len: int):
    return dsrc.databunch(bs=bs, seq_len=seq_len, after_batch=[Cuda()], dl_type=LMDataLoader)

dbunch = get_databunch(dsrc, bs=2, seq_len=seq_len)
# for x, y in dbunch.train_dl:
#     print(x.shape, x.dtype)
#     print(x.shape, y.dtype)
#     break

In [ ]:
dbunch.show_batch(max_n=2)

,text,text_
0,"= Spanish Hill = \n \n Spanish Hill is a hill located in the borough of South Waverly, Pennsylvania. The hill is controversial among historians and archaeologists ; opinions regarding the origin of structures found on the site vary from embankments created by early farmers, to the remnants of a Native American village and <unk>, due to the site '","Spanish Hill = \n \n Spanish Hill is a hill located in the borough of South Waverly, Pennsylvania. The hill is controversial among historians and archaeologists ; opinions regarding the origin of structures found on the site vary from embankments created by early farmers, to the remnants of a Native American village and <unk>, due to the site's"
1,"his influence to have Gladstone invited to stand as parliamentary candidate for Midlothian, near to Rosebery's Dalmeny estate. Gladstone had nominally retired from politics after losing his Greenwich seat in 1874, when Disraeli had been swept to power. The campaign was based at Dalmeny where Lady Rosebery hosted a series of","influence to have Gladstone invited to stand as parliamentary candidate for Midlothian, near to Rosebery's Dalmeny estate. Gladstone had nominally retired from politics after losing his Greenwich seat in 1874, when Disraeli had been swept to power. The campaign was based at Dalmeny where Lady Rosebery hosted a series of large"


## Learner

In [ ]:
dbunch = get_databunch(dsrc, bs=bs, seq_len=seq_len)
model = AutoModelWithLMHead.from_pretrained(model_name)
learn = Learner(dbunch, 
                model, 
                loss_func=CrossEntropyLossFlat(), 
                opt_func=ranger,
                splitter=gpt2_lmhead_split, 
                cbs=[GPT2LMHeadCallback()],
                metrics=[accuracy, Perplexity()],
               )

In [ ]:
learn.freeze_to(-1)
# Seems like my gpt2_lmhead_split is useless for lm_head but I don't know why, so I manually unfreeze the last layer.
learn.model.lm_head.requires_grad_(True)
learn.summary()

GPT2LMHeadModel (Input shape: 64 x 72)
Layer (type)         Output Shape         Param #    Trainable 
Embedding            64 x 72 x 768        38,597,376 True      
________________________________________________________________
Embedding            64 x 72 x 768        786,432    False     
________________________________________________________________
Dropout              64 x 72 x 768        0          False     
________________________________________________________________
LayerNorm            64 x 72 x 768        1,536      False     
________________________________________________________________
Conv1D               64 x 72 x 2304       1,771,776  False     
________________________________________________________________
Conv1D               64 x 72 x 768        590,592    False     
________________________________________________________________
Dropout              64 x 12 x 72 x 72    0          False     
___________________________________________________________

In [ ]:
learn.fit_one_cycle(1, 1e-2, moms=(0.8,0.7,0.8), cbs=[ShortEpochCallback(short_valid=False)])

In [ ]:
# learn.lr_find()

In [ ]:
# learn.fit_one_cycle(2, 1e-2, moms=(0.8,0.7,0.8))

In [ ]:
# learn.save(f'{model_name}_1')

In [ ]:
# # learn.load(f'{model_name}_1')
# learn.freeze_to(-2)
# # Seems like my gpt2_lmhead_split is useless for lm_head but I don't know why, so I manually unfreeze the last layer.
# learn.model.lm_head.requires_grad_(True)
# learn.fit_one_cycle(2, 1e-3, moms=(0.8,0.7,0.8))

In [ ]:
# learn.save(f'{model_name}_2')

## Export

In [ ]:
# # fastai model
# learn.save(f'{model_name}_final')

In [ ]:
# # transformeres model
# Path('./models/transformers_model').mkdir(exist_ok=True)
# learn.model.save_pretrained('./models/transformers_model')

In [ ]:
# Path('./models/tokenizer').mkdir(exist_ok=True)
# tokenizer.save_pretrained('./models/tokenizer')

## Client inference

In [ ]:
# inference_model = GPT2LMHeadModel.from_pretrained('./models/transformers_model')
# inference_model.eval()
# tokenizer = GPT2Tokenizer.from_pretrained('./models/tokenizer')

In [ ]:
# test_sentences = ['The dog', 'I think we']

In [ ]:
# num_return_sequences = 2
# with torch.no_grad():
#     for sentence in test_sentences:
#         numeric_sentence = tokenizer.encode(sentence)
#         x = torch.tensor([numeric_sentence])
#         outputs = inference_model.generate(x, max_length=20, do_sample=True, num_beams=5, num_return_sequences=num_return_sequences, temperature=1.5)
#         for j in range(num_return_sequences): #  3 output sequences were generated
#             print('Generated {}: {}'.format(j, tokenizer.decode(outputs[0][j])))
#         print()